<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #5: TF-IDF Jhonathan Giraldo
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Punto 1: Pre-Procesamiento

- `[18 pts]` Leer el archivo `Princesas.csv` usando `pandas` y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [13]:
data = pd.read_csv('C:/Users/jagir/OneDrive/Konrad/NPL/ProyectoNLP/Clase12092020/Princesas.csv')
data = pd.DataFrame(data)
data

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [10]:
def preprocesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ",texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto   

In [14]:
data['pre-procesado']=data['Personalidad'].apply(lambda texto: preprocesado(texto))
data

,Princesa,Personalidad,pre-procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancanieves princesa noble cuna ve forzada se...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicienta inicialmente sirvienta casa constan...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princesa aurora bella durmiente hija única rei...
3,Bella,Bella es una muchacha que vive en la campiña f...,bella muchacha vive campiña francesa padre inv...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introdujo primera vez princesa jasmín pocos dí...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombre pocahontas significa pequeña silenciosa...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mulan atípica anteriores papeles femeninos pel...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,joven camarera sueña ser dueña propio restaura...
8,Mérida,Mérida llama la atención por su característico...,mérida llama atención característico pelo anar...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moana joven años edad hija única sucesora impo...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(data['pre-procesado'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())## Lo envia a un df con los nombres 
tfidf_matrix.index = data['Princesa']

tfidf_mat = tfidf_matrix.T.round(3)
tfidf_mat

Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrumada,0.000,0.0,0.066,0.0,0.0,0.000,0.000,0.000,0.000,0.000
abuela,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.102
acuerdo,0.000,0.0,0.056,0.0,0.0,0.000,0.000,0.000,0.083,0.000
además,0.000,0.0,0.000,0.0,0.0,0.098,0.000,0.000,0.000,0.000
afortunados,0.087,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
waka,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.102
área,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.098,0.000
época,0.000,0.0,0.000,0.0,0.0,0.000,0.123,0.000,0.000,0.000
ésta,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.133,0.000,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de las princesas
- `[2 pts]` ¿Cuáles son las princesas más parecidas?
- `[2 pts]` ¿Cuáles son las princesas más diferentes?

In [69]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_mat.T.values)
dist_cos = pd.DataFrame(dist_cos, columns =  tfidf_mat.columns, index = tfidf_mat.columns)
dist_cos


Princesa,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Princesa,,,,,,,,,,
Blancanieves,0.000000,0.866797,0.894826,0.924564,0.943613,0.938101,0.902638,0.977888,0.989218,0.968750
Cenicienta,0.866797,0.000000,0.923960,0.959074,0.978082,0.973134,0.963874,0.940412,0.993106,0.984062
Aurora,0.894826,0.923960,0.000000,0.879406,0.949429,0.915074,0.922912,0.963668,0.964094,0.972084
Bella,0.924564,0.959074,0.879406,0.000000,0.893927,0.912105,0.898804,0.980095,0.986298,0.943764
Jasmín,0.943613,0.978082,0.949429,0.893927,0.000000,0.944627,0.974709,0.956344,0.968981,0.967041
Pocahontas,0.938101,0.973134,0.915074,0.912105,0.944627,0.000000,0.918213,0.966241,0.979011,0.935330
Mulan,0.902638,0.963874,0.922912,0.898804,0.974709,0.918213,0.000000,0.990616,0.987046,0.956046
Tiana,0.977888,0.940412,0.963668,0.980095,0.956344,0.966241,0.990616,0.000000,0.986056,0.996094
Mérida,0.989218,0.993106,0.964094,0.986298,0.968981,0.979011,0.987046,0.986056,0.000000,0.988911


## Respuesta 1 :
#### Las mas parecidas son 
#### cenicienta y blancanieves = 0,866797	
#### aurora y blancanieves = 0,894826	
#### bella y aurora = 0,879406
		

## Respuesta 2 :
#### Las mas diferentes son 
#### tiana y mulan = 0,990616
#### Merida y cenicienta = 0,993106	
#### moana y tiana = 0,996094